In [1]:
import requests
from bs4 import BeautifulSoup
import sqlite3

In [5]:
# Подключаемся к БД
#conn = sqlite3.connect('C:/app/bd_apt.db')
#cursor1 = conn.cursor()
#cursor2 = conn.cursor()
#cursor3 = conn.cursor()

# Делаем запрос и получаем страницу
url = 'https://apteka.ru/catalog/section/lekarstvennye-preparaty/'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
ART = soup.find_all('h2')
x = []
# Ищем все относительные ссылки на группы лекарств
for i in ART:
    a = i.find('a')
    if a is None:
        break
    ah = a.get('href')
    print(ZZZ, ah)
    ZZZ += 1
    x.append(ah)
x1=0

1 /catalog/section/lekarstvennye-preparaty/antibakterialnye-preparaty/
2 /catalog/section/lekarstvennye-preparaty/antioksidantnye-sredstva/
3 /catalog/section/lekarstvennye-preparaty/antipodagricheskie-sredstva/
4 /catalog/section/lekarstvennye-preparaty/antisepticheskie-sredstva/
5 /catalog/section/lekarstvennye-preparaty/vrednye-privychki/
6 /catalog/section/lekarstvennye-preparaty/gomeopaticheskie-sredstva/
7 /catalog/section/lekarstvennye-preparaty/gormonalnye-preparaty/
8 /catalog/section/lekarstvennye-preparaty/dermatologicheskie-preparaty/
9 /catalog/section/lekarstvennye-preparaty/drugie-lechebnye-sredstva/
10 /catalog/section/lekarstvennye-preparaty/dykhatelnaya-sistema/
11 /catalog/section/lekarstvennye-preparaty/meditsinskie-immunobiologicheskie-preparaty-mibp/
12 /catalog/section/lekarstvennye-preparaty/immunnaya-sistema/
13 /catalog/section/lekarstvennye-preparaty/rastvory-dlya-parenteralnogo-pitaniya/
14 /catalog/section/lekarstvennye-preparaty/zabolevaniya-kostey/
15 /ca

In [116]:
def sel_a():
    cursor3.execute("""SELECT * FROM activ """)
    results3 = cursor3.fetchall()

In [117]:
def sel_c():
    cursor2.execute("""SELECT * FROM contry """)
    results2 = cursor2.fetchall()

In [123]:
# Проходим по страницам товраов и получаем ссылки на страницу группы товаров
def per(url_group, col):
    sel_a()
    sel_c()
    i = 1
    while i<= col:
        url = url_group + "?PAGEN_1="
        url += str(i)
        page = requests.get(url)
        beaut(page)
        print(url)
        i += 1

In [122]:
# Парсим каждую страницу и получаес ссылкии на товар
def beaut(page):
    n = 0
    soup = BeautifulSoup(page.content, 'html.parser')
    ART = soup.find_all('article')
    for i in ART:
        prod = i.find('div', class_='item_name item_name-catalog')
        a = prod.find('a')
        link_on_med = a.get('href')
        n += 1
        page_medicament(link_on_med)

In [118]:
# Парсинг страницы описания товара лекарственного средства
def page_medicament(link_on_med):
    price = ''
    activ = ''
    contry = ''
    url_med = aurl + link_on_med
    page = requests.get(url_med)
    soup = BeautifulSoup(page.content, 'html.parser')
    h1_name = soup.find('h1')
    price_prom = soup.find('div', class_='item_price item_price-card')
    
    try:
        price = price_prom.find('span')
        medicament = soup.find_all('tr')
        for i in medicament:
            t = i.find_all('td')
            if t[0].text == 'Действующие вещества:':
                activ = t[1].text
            if t[0].text == 'Страна происхождения:':
                contry = t[1].text
            #print(t[0],t[1])
        sel_a()
        sel_c()
        add_bd(h1_name.text, price.text, activ, contry)
    except Exception:
        x1=+1

In [119]:
# Добавление в таблицу названия препарата,
# а если нужно то и вызов функций добавления его активного вещества и страну производства
def add_bd(name, price, activ, contry):
    naz_a_id = '1'
    activ_id = ''
    vol = ''
    contry_id = ''
    for x in results3:
        if activ == x[1]:
            activ_id = x[0]
    if activ_id == '':
        activ_id = add_activ(activ)
        
    for x in results2:
        if x[1] == contry:
            contry_id = x[0]
    if contry_id == '':
        contry_id = add_con(contry)
    
    mass = name.split(' ')
    for y in range(len(mass)):
        if y != 0:
            vol += mass[y]
    cursor1.execute("""INSERT INTO lec (name, volume,price,naz_a_id,contry_id,aktiv_id) 
    VALUES (?, ?, ?, ?, ?, ?)""", 
                    (mass[0], vol, price, naz_a_id, str(contry_id), str(activ_id)))
    conn.commit()

In [120]:
def add_activ(activ):
    cursor3.execute("""INSERT INTO activ (name) VALUES (?)""", (activ,))
    conn.commit()
    
    cursor3.execute("""SELECT MAX(id) FROM activ """)
    chislo = cursor3.fetchall()
    
    sel_a()
    return chislo

In [121]:
def add_con(country):
    cursor2.execute("""INSERT INTO contry (name) VALUES (?)""", (country,))
    conn.commit()
    
    cursor2.execute("""SELECT MAX(id) FROM contry """)
    chislo = cursor2.fetchall()
    
    sel_c()
    return chislo

In [124]:
# Перебираем группы лекарств и получаем url и кол-во вкладок в группе
aurl='https://apteka.ru'
results2
results3
for i in range(len(x)):
    url_group = aurl + x[i]
    page = requests.get(uurl)

    soup = BeautifulSoup(page.content, 'html.parser')
    ART = soup.find('div', class_="items catalog-items js-products-container")
    lin = ART.find_all('li',class_="")
    col = 0
    for i in lin:
        a = i.find('a',class_="link page element")
        try:
            if col < int(a.text):
                col = int(a.text)
        except Exception:
            pass
    if col == 0:
        col = 1
    per(url_group, col)

https://apteka.ru/catalog/section/lekarstvennye-preparaty/antibakterialnye-preparaty/?PAGEN_1=1
https://apteka.ru/catalog/section/lekarstvennye-preparaty/antibakterialnye-preparaty/?PAGEN_1=2
https://apteka.ru/catalog/section/lekarstvennye-preparaty/antibakterialnye-preparaty/?PAGEN_1=3
https://apteka.ru/catalog/section/lekarstvennye-preparaty/antibakterialnye-preparaty/?PAGEN_1=4
https://apteka.ru/catalog/section/lekarstvennye-preparaty/antibakterialnye-preparaty/?PAGEN_1=5
https://apteka.ru/catalog/section/lekarstvennye-preparaty/antibakterialnye-preparaty/?PAGEN_1=6
https://apteka.ru/catalog/section/lekarstvennye-preparaty/antibakterialnye-preparaty/?PAGEN_1=7
https://apteka.ru/catalog/section/lekarstvennye-preparaty/antibakterialnye-preparaty/?PAGEN_1=8
https://apteka.ru/catalog/section/lekarstvennye-preparaty/antibakterialnye-preparaty/?PAGEN_1=9
https://apteka.ru/catalog/section/lekarstvennye-preparaty/antibakterialnye-preparaty/?PAGEN_1=10
https://apteka.ru/catalog/section/lekar

https://apteka.ru/catalog/section/lekarstvennye-preparaty/immunnaya-sistema/?PAGEN_1=1
https://apteka.ru/catalog/section/lekarstvennye-preparaty/immunnaya-sistema/?PAGEN_1=2
https://apteka.ru/catalog/section/lekarstvennye-preparaty/immunnaya-sistema/?PAGEN_1=3
https://apteka.ru/catalog/section/lekarstvennye-preparaty/immunnaya-sistema/?PAGEN_1=4
https://apteka.ru/catalog/section/lekarstvennye-preparaty/immunnaya-sistema/?PAGEN_1=5
https://apteka.ru/catalog/section/lekarstvennye-preparaty/immunnaya-sistema/?PAGEN_1=6
https://apteka.ru/catalog/section/lekarstvennye-preparaty/immunnaya-sistema/?PAGEN_1=7
https://apteka.ru/catalog/section/lekarstvennye-preparaty/rastvory-dlya-parenteralnogo-pitaniya/?PAGEN_1=1
https://apteka.ru/catalog/section/lekarstvennye-preparaty/rastvory-dlya-parenteralnogo-pitaniya/?PAGEN_1=2
https://apteka.ru/catalog/section/lekarstvennye-preparaty/rastvory-dlya-parenteralnogo-pitaniya/?PAGEN_1=3
https://apteka.ru/catalog/section/lekarstvennye-preparaty/zabolevaniya

https://apteka.ru/catalog/section/lekarstvennye-preparaty/pishchevaritelnyy-trakt-i-obmen-veshchestv/?PAGEN_1=4
https://apteka.ru/catalog/section/lekarstvennye-preparaty/pishchevaritelnyy-trakt-i-obmen-veshchestv/?PAGEN_1=5
https://apteka.ru/catalog/section/lekarstvennye-preparaty/pishchevaritelnyy-trakt-i-obmen-veshchestv/?PAGEN_1=6
https://apteka.ru/catalog/section/lekarstvennye-preparaty/pishchevaritelnyy-trakt-i-obmen-veshchestv/?PAGEN_1=7
https://apteka.ru/catalog/section/lekarstvennye-preparaty/pishchevaritelnyy-trakt-i-obmen-veshchestv/?PAGEN_1=8
https://apteka.ru/catalog/section/lekarstvennye-preparaty/pishchevaritelnyy-trakt-i-obmen-veshchestv/?PAGEN_1=9
https://apteka.ru/catalog/section/lekarstvennye-preparaty/pishchevaritelnyy-trakt-i-obmen-veshchestv/?PAGEN_1=10
https://apteka.ru/catalog/section/lekarstvennye-preparaty/pishchevaritelnyy-trakt-i-obmen-veshchestv/?PAGEN_1=11
https://apteka.ru/catalog/section/lekarstvennye-preparaty/pishchevaritelnyy-trakt-i-obmen-veshchestv/?

https://apteka.ru/catalog/section/lekarstvennye-preparaty/rastitelnye-naturalnye-sr-va/?PAGEN_1=10
https://apteka.ru/catalog/section/lekarstvennye-preparaty/rastitelnye-naturalnye-sr-va/?PAGEN_1=11
https://apteka.ru/catalog/section/lekarstvennye-preparaty/rastitelnye-naturalnye-sr-va/?PAGEN_1=12
https://apteka.ru/catalog/section/lekarstvennye-preparaty/rastitelnye-naturalnye-sr-va/?PAGEN_1=13
https://apteka.ru/catalog/section/lekarstvennye-preparaty/rastitelnye-naturalnye-sr-va/?PAGEN_1=14
https://apteka.ru/catalog/section/lekarstvennye-preparaty/rastitelnye-naturalnye-sr-va/?PAGEN_1=15
https://apteka.ru/catalog/section/lekarstvennye-preparaty/serdechno-sosudistaya-sistema/?PAGEN_1=1
https://apteka.ru/catalog/section/lekarstvennye-preparaty/serdechno-sosudistaya-sistema/?PAGEN_1=2
https://apteka.ru/catalog/section/lekarstvennye-preparaty/serdechno-sosudistaya-sistema/?PAGEN_1=3
https://apteka.ru/catalog/section/lekarstvennye-preparaty/serdechno-sosudistaya-sistema/?PAGEN_1=4
https://ap

In [113]:
x1

0